In [1]:
%%sql

-- Apaga a tabelas se existirem
DROP TABLE IF EXISTS tb_log_eventos;
DROP TABLE IF EXISTS tb_funcionarios;

-- Cria a tabela 'tb_log_eventos'
CREATE TABLE IF NOT EXISTS tb_log_eventos(
    Matricula INT, 
    Nome VARCHAR(200),
    Filial VARCHAR(100),
    Departamento VARCHAR(150),
    Cargo VARCHAR(150),
    DataAdmissao DATE,
    DataDesligamento DATE,
    Operacao STRING,
    DataLog TIMESTAMP
)
USING DELTA;

StatementMeta(, fbb0e6a2-533a-4ca7-914c-e2c97c967bef, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
%%sql

-- Insere dados de exemplo na 'tb_log_eventos'
INSERT INTO tb_log_eventos(Matricula, Nome, Filial, Departamento, Cargo, DataAdmissao, DataDesligamento, Operacao, DataLog) VALUES 
(98, 'FLAVIO FAGUNDES', 'JUNDIAI', 'PRODUCAO', 'OPERADOR DE MAQUINA III', '2021-12-07', NULL, 'ADMISSAO', '2021-12-07 08:40:23'),
(100, 'MARIA PEREIRA', 'JUNDIAI', 'PRODUCAO', 'AJUDANTE DE PRODUCAO', '2022-01-17', NULL, 'ADMISSAO', '2022-01-17 09:10:17'),
(143, 'JOSE DA SILVA', 'JUNDIAI', 'PRODUCAO', 'OPERADOR DE MAQUINA III', '2022-02-05', NULL, 'ADMISSAO', '2022-02-05 13:00:00'),
(100, 'MARIA PEREIRA', 'JUNDIAI', 'PRODUCAO', 'OPERADOR DE MAQUINA I', '2022-01-17', NULL, 'ALTERACAO', '2023-02-20 15:00:00'),
(98, 'FLAVIO FAGUNDES', 'CAMPINAS', 'PRODUCAO', 'OPERADOR DE MAQUINA III', '2021-12-07', NULL, 'ALTERACAO', '2023-03-01 14:00:00'),
(143, 'JOSE DA SILVA', 'JUNDIAI', 'MANUTENCAO', 'ELETRICISTA DE MANUTENCAO PL', '2022-02-05', NULL, 'ALTERACAO', '2023-04-07 15:00:00'),
(202, 'MATEUS SOUZA', 'PIRACICABA', 'PRODUCAO', 'OPERADOR DE MAQUINA I', '2023-04-29', NULL, 'ADMISSAO', '2023-04-29 10:00:00'),
(298, 'MARCOS TEIXEIRA', 'CAMPINAS', 'QUALIDADE', 'ANALISTA DE QUALIDADE JR', '2023-07-14', NULL, 'ADMISSAO', '2023-07-14 15:00:00'),
(100, 'MARIA PEREIRA', 'JUNDIAI', 'PRODUCAO', 'OPERADOR DE MAQUINA II', '2022-01-17', NULL, 'ALTERACAO', '2024-03-05 08:00:00'),
(202, 'MATEUS SOUZA', 'CAMPINAS', 'PRODUCAO', 'OPERADOR DE MAQUINA II', '2023-04-29', NULL, 'ALTERACAO', '2024-09-19 11:00:00'),
(100, 'MARIA PEREIRA', 'JUNDIAI', 'PRODUCAO', 'OPERADOR DE MAQUINA III', '2022-01-17', NULL, 'ALTERACAO', '2024-10-10 09:00:00'),
(98, 'FLAVIO FAGUNDES', 'CAMPINAS', 'PRODUCAO', 'SUPERVISOR DE PRODUCAO', '2021-12-07', NULL, 'ALTERACAO', '2024-11-05 10:00:00'),
(298, 'MARCOS TEIXEIRA', 'PIRACICABA', 'QUALIDADE', 'ANALISTA DE QUALIDADE JR', '2023-07-14', '2024-11-10', 'DESLIGAMENTO', '2024-11-10 08:00:00'),
(100, 'MARIA PEREIRA', 'JUNDIAI', 'PRODUCAO', 'OPERADOR DE MAQUINA III', '2022-01-17', '2024-12-13', 'DESLIGAMENTO', '2024-12-13 13:00:00'),
(143, 'JOSE DA SILVA', 'JUNDIAI', 'MANUTENCAO', 'ELETRICISTA DE MANUTENCAO SR', '2022-02-05', NULL, 'ALTERACAO', '2024-12-15 09:00:00');


StatementMeta(, fbb0e6a2-533a-4ca7-914c-e2c97c967bef, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql

-- Criação da tabela 'tb_funcionario' a partir do 'tb_log_eventos' 

-- Cria ou subtitui a tabela 'tb_funcionarios'
CREATE OR REPLACE TABLE tb_funcionarios USING DELTA AS 

-- CTE com o log apenas das linhas onde 'Operacao' não é 'DESLIGAMENTO'
WITH CTE_Log AS (
    SELECT 
        Matricula,
        Nome, 
        Filial,
        Departamento,
        Cargo,
        DataAdmissao,
        DataDesligamento,
        DataLog,
        DataLog AS DataVigenciaInicial,
        
        -- Window function para trazer o próximo 'DataLog' por 'Matricula'
        LEAD(DataLog) OVER (PARTITION BY Matricula ORDER BY DataLog) AS ProximoDataLog,
        
        -- Window function para criar a coluna 'MatriculaSk' a partir da ordem do 'DataLog'
        ROW_NUMBER() OVER (ORDER BY DataLog) AS MatriculaSk
    
    FROM tb_log_eventos
    WHERE Operacao <> 'DESLIGAMENTO'
)

-- Seleciona a partir da 'CTE_Log'
SELECT 
    MatriculaSk,  
    Matricula,
    Nome, 
    Filial,
    Departamento,
    Cargo,
    DataAdmissao,
    DataDesligamento,
    DataLog,

    -- 'DataVigenciaInicial' é a data de alteração
    CAST(DataVigenciaInicial AS DATE) AS DataVigenciaInicial, 
    
    -- Coluna 'DataVigenciaFinal'
    -- Quando o 'ProximoDataLog' não estiver vazio, pega o
    -- 'ProximoDataLog' e subtrai um dia
    -- Caso estiver vazio então quer dizer que é o último
    -- Logo adiciona '9999-12-31'
    CASE 
        WHEN ProximoDataLog IS NOT NULL 
        THEN CAST(DATEADD(DAY, -1, ProximoDataLog) AS DATE)
        ELSE CAST('9999-12-31' AS DATE)
    END AS DataVigenciaFinal,
    
    -- 'EstaAtivo' traz 1 quando for o último evento
    CASE 
        WHEN ProximoDataLog IS NULL THEN 1 
        ELSE 0 
    END AS EstaAtivo

FROM CTE_Log
ORDER BY DataLog;


StatementMeta(, fbb0e6a2-533a-4ca7-914c-e2c97c967bef, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql

-- Update das operações de desligamento

-- Cria CTE com as operações de desligamento
WITH CTE_Desligamentos AS (
    SELECT 
        Matricula,
        Nome, 
        Filial,
        Departamento,
        Cargo,
        DataAdmissao,
        DataDesligamento,
        DataLog,
        CAST(DataLog AS DATE) AS DataVigenciaInicial
    FROM tb_log_eventos
    WHERE Operacao = 'DESLIGAMENTO'
)

-- Realiza o merge na tabela 'tb_funcionarios'
MERGE INTO tb_funcionarios AS f

-- a partir da 'CTE_Desligamentos'
USING (SELECT * FROM CTE_Desligamentos) AS d  

-- nas linhas onde as matriculas correspondem e estão ativos
ON f.Matricula = d.Matricula AND f.EstaAtivo = 1

-- Atualizando os registros que já existem
WHEN MATCHED THEN 
    UPDATE SET 

        -- As colunas 'DataDesligamento' e 'DataVigenciaFinal' em tb_funcionarios
        -- recebem o dia anterior da 'DataVigenciaInicial' da 'CTE_Desligamentos'
        f.DataDesligamento = CAST(DATEADD(DAY, -1, d.DataVigenciaInicial) AS DATE),
        f.DataVigenciaFinal = CAST(DATEADD(DAY, -1, d.DataVigenciaInicial) AS DATE),

        -- E coloca 'EstaAtivo' como 0
        f.EstaAtivo = 0;


StatementMeta(, fbb0e6a2-533a-4ca7-914c-e2c97c967bef, 7, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [5]:
%%sql 

-- Exibe os resultados carregados em 'tb_funcionarios'
SELECT * FROM tb_funcionarios
ORDER BY DataLog;

StatementMeta(, fbb0e6a2-533a-4ca7-914c-e2c97c967bef, 8, Finished, Available, Finished)

<Spark SQL result set with 13 rows and 12 fields>